In [8]:
import os, sys, glob, time, pdb
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.cm
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Pool
from importlib import reload
import socket

In [9]:
hostname = socket.gethostname()
print(f'hostname: {hostname}')
print(f"conda env: {os.environ['CONDA_DEFAULT_ENV']}")
data_root = '/gpfs/alpine/syb105/proj-shared/Projects/GeoBio_CoMet/data'
out_root = '/gpfs/alpine/syb105/proj-shared/Personal/jmerlet/projects/sars_cov_2_geo'

hostname: andes107.olcf.ornl.gov
conda env: /gpfs/alpine/syb105/proj-shared/Personal/jmerlet/envs/conda/andes/andes_matplotlib


In [3]:
# load metadata
meta_path = os.path.join(data_root, 'metadata/metadata_2022_06_02/metadata.tsv')
meta = pd.read_csv(meta_path, sep='\t', low_memory=False)
total_entries = len(meta)

In [4]:
meta.columns

Index(['Virus name', 'Type', 'Accession ID', 'Collection date', 'Location',
       'Additional location information', 'Sequence length', 'Host',
       'Patient age', 'Gender', 'Clade', 'Pango lineage', 'Pangolin version',
       'Variant', 'AA Substitutions', 'Submission date', 'Is reference?',
       'Is complete?', 'Is high coverage?', 'Is low coverage?', 'N-Content',
       'GC-Content'],
      dtype='object')

## Gender

In [38]:
gender = meta.loc[:, 'Gender']
gender = np.char.lower(gender.values.astype(str))

In [44]:
uniq_genders, count_genders = np.unique(gender, return_counts=True)
idx = np.argsort(count_genders)[::-1]
uniq_genders = uniq_genders[idx]
counts_genders = count_genders[idx]

In [45]:
uniq_genders[:10]

array(['unknown', 'female', 'male', 'unknow', 'nan', 'ambiguous',
       'missing', 'unkown', 'not provided', 'unkmown'], dtype='<U29')

In [46]:
counts_genders[:10]

array([7017212, 2133501, 1983003,    3169,    2891,    1179,     922,
           768,     727,     590])

## Age

In [51]:
age = meta.loc[:, 'Patient age']
age_str = age[:, 

In [54]:
age

0           unknown
1           unknown
2           unknown
3           unknown
4           unknown
             ...   
11152776    unknown
11152777    unknown
11152778    unknown
11152779    unknown
11152780    unknown
Name: Patient age, Length: 11152781, dtype: object

In [53]:
uniq_ages, count_ages = np.unique(age, return_counts=True)
idx = np.argsort(count_ages)[::-1]
uniq_ages = uniq_ages[idx]
counts_ages = count_ages[idx]

TypeError: '<' not supported between instances of 'float' and 'str'

## Wuhan-1 in Wuhan cluster

In [14]:
clusters_path = os.path.join(data_root, 'aligned/sequences_2022_06_02/uniq_ids/preprocessed_d-cutoff_1000_n-cutoff_0.01_pos_342-29665/ns-as-0s_mutation_count_filtered_100/thresh_run_HH-0.19991/hip_mcl/clusters_inflation-1.2.tsv')

In [18]:
lgst_wuhan_cluster_idx = 0
lgst_wuhan_cluster_size = 0

with open(clusters_path) as in_file:
    for i, line in enumerate(in_file):
        seqs = line.strip().split(' ')
        num_seqs = len(seqs)
        if '000000000_T' in seqs or '000000000_A' in seqs:
            if num_seqs > lgst_wuhan_cluster_size:
                lgst_wuhan_cluster_size = num_seqs
                lgst_wuhan_cluster_idx = i
                
print(lgst_wuhan_cluster_idx)
print(lgst_wuhan_cluster_size)

43
11489


In [19]:
with open(clusters_path) as in_file:
    for i, line in enumerate(in_file):
        if i != lgst_wuhan_cluster_idx:
            continue
        else:
            wuhan = line.strip().split(' ')

In [23]:
# getting similarity scores for wuhan not wuhan
edge_values_path = os.path.join(data_root, 'aligned/sequences_2022_06_02/uniq_ids/preprocessed_d-cutoff_1000_n-cutoff_0.01_pos_342-29665/ns-as-0s_mutation_count_filtered_100/thresh_run_HH-0.19991/combined_all.tsv')

In [ ]:
identical_seqs = []
with open(edge_values_path) as in_file:
    for i, line in enumerate(in_file):
        seq_a, seq_b, score = line.strip().split('\t')
        if seq_a == '000000000_T':
            if score == 2.0:
                identical_seqs.append(seq_b)
        elif seq_b == '000000000_T':
            if score == 2.0:
                identical_seqs.append(seq_a)
        if i % 100000000000 == 0: print(i)    

0
